# **XGBoost**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 6 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install xgboost

In [3]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.0 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/NotAnnotated/Regular/AveragePooled/split_features_6s_all_1D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [5]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [6]:
train_data

{'melspectrogram': array([[-0.01776731, -0.989827  , -0.99071366, ..., -0.8216415 ,
         -0.85376334, -0.9824129 ],
        [-0.01776753, -0.9898328 , -0.99071616, ..., -0.8170307 ,
         -0.84770477, -0.9777302 ],
        [-0.01776784, -0.989846  , -0.99073434, ..., -0.82816565,
         -0.8608326 , -0.98463976],
        ...,
        [-0.01781281, -0.5766479 , -0.54141074, ..., -0.87102264,
         -0.89623916, -0.99998677],
        [-0.01774438, -0.57157916, -0.5401701 , ..., -0.8684321 ,
         -0.89438707, -0.99997807],
        [-0.01779194, -0.5788874 , -0.54096264, ..., -0.8674551 ,
         -0.89504766, -0.999996  ]], dtype=float32),
 'mfcc': array([[-1.        , -0.10011531, -0.14837739, ...,  0.02309681,
          0.0204311 ,  0.04844239],
        [-1.        , -0.11325053, -0.18573974, ...,  0.02437621,
          0.02035097,  0.05108224],
        [-1.        , -0.10374827, -0.18394189, ...,  0.02396246,
          0.01970905,  0.04918307],
        ...,
        [-0.9

In [7]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [8]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [9]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [10]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [11]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [12]:
display(train_y.shape)
display(train_y[:15])

(6543,)

array([19,  2,  7,  1,  2, 19,  7,  7,  0, 18, 19, 12,  0, 15, 12],
      dtype=int32)

In [13]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(6543, 128)

array([-0.01776706, -0.98747253, -0.982526  , -0.96910095, -0.942215  ,
       -0.9638503 , -0.9875802 , -0.9905174 , -0.9911741 , -0.99101436,
       -0.9906444 , -0.9879602 , -0.9845477 , -0.97809017, -0.9798955 ,
       -0.98365515, -0.9822163 , -0.97887224, -0.98426366, -0.98790485,
       -0.9786248 , -0.9810127 , -0.98267984, -0.98049825, -0.98332447,
       -0.9814118 , -0.981292  , -0.9789223 , -0.9792321 , -0.98115015,
       -0.98421574, -0.9866537 , -0.9838543 , -0.9709954 , -0.9539507 ,
       -0.94942206, -0.9324423 , -0.9088786 , -0.89429635, -0.87762254,
       -0.86245364, -0.83916736, -0.80800724, -0.7875475 , -0.7559936 ,
       -0.72916424, -0.7109448 , -0.6965236 , -0.68793595, -0.67658234,
       -0.6674314 , -0.65027046, -0.63032126, -0.6145561 , -0.5971278 ,
       -0.5862773 , -0.5772054 , -0.5720244 , -0.574622  , -0.57558393,
       -0.5744448 , -0.5712204 , -0.56524533, -0.5525866 , -0.5526075 ,
       -0.5596768 , -0.5844143 , -0.60527414, -0.62253195, -0.63

'mfcc'

(6543, 20)

...

(6543, 84)

array([-73.09822 , -73.197426, -73.273796, -73.76968 , -73.58266 ,
       -73.19705 , -73.832115, -74.14205 , -73.916565, -73.89668 ,
       -73.674866, -73.44959 , -73.76412 , -73.24343 , -73.15452 ,
       -73.37146 , -72.83074 , -71.93702 , -71.83897 , -72.02708 ,
       -71.25811 , -70.759285, -68.80554 , -68.4736  , -67.8511  ,
       -68.15875 , -69.84771 , -71.37478 , -72.96278 , -73.65958 ,
       -74.23347 , -74.917534, -75.76558 , -75.92188 , -75.96454 ,
       -75.89953 , -76.31836 , -76.23387 , -75.3822  , -74.438065,
       -73.990456, -73.06567 , -72.99704 , -73.46609 , -74.20816 ,
       -74.00004 , -73.73749 , -74.57999 , -74.83605 , -74.46016 ,
       -74.98341 , -75.22031 , -75.50244 , -75.54683 , -75.75173 ,
       -76.298225, -76.15754 , -73.803116, -71.55328 , -68.58164 ,
       -64.66023 , -60.468758, -56.253662, -52.7704  , -50.42955 ,
       -48.130302, -45.72306 , -43.541336, -42.784485, -42.18103 ,
       -41.757645, -40.58786 , -41.421165, -44.19097 , -46.699

'id'

(6543,)

'XC280984.mp3'

In [14]:
display(val_y.shape)
display(val_y[:15])

(2028,)

array([12,  7, 19,  3,  7, 13,  9, 11, 11, 12, 17,  2, 17,  3, 18],
      dtype=int32)

In [15]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(2028, 128)

array([-0.01777715, -0.6823705 , -0.6700081 , -0.7143769 , -0.72569156,
       -0.7217254 , -0.70796573, -0.7020234 , -0.6957035 , -0.69550025,
       -0.68860817, -0.6916205 , -0.694039  , -0.70643896, -0.7069956 ,
       -0.7030926 , -0.6921738 , -0.69109046, -0.6968512 , -0.69173974,
       -0.6809189 , -0.68640286, -0.6812207 , -0.68139845, -0.67214686,
       -0.67690176, -0.675241  , -0.6680333 , -0.6716892 , -0.6699464 ,
       -0.66820323, -0.66705185, -0.6613608 , -0.66557056, -0.6653173 ,
       -0.6774219 , -0.6719049 , -0.6611124 , -0.66049206, -0.6563231 ,
       -0.6591102 , -0.64467436, -0.64172274, -0.638755  , -0.6407541 ,
       -0.6419572 , -0.6475931 , -0.6344285 , -0.6343891 , -0.64334345,
       -0.64528894, -0.63966703, -0.6269635 , -0.6387499 , -0.6530332 ,
       -0.6511723 , -0.6597135 , -0.66876924, -0.68288004, -0.6819005 ,
       -0.68560714, -0.68490344, -0.6785722 , -0.67853594, -0.6710221 ,
       -0.6644165 , -0.64187586, -0.621178  , -0.6261929 , -0.64

'mfcc'

(2028, 20)

...

(2028, 84)

array([-37.927734, -37.442028, -40.543606, -39.191135, -38.423588,
       -39.843575, -41.259094, -38.99902 , -39.11001 , -40.62095 ,
       -41.056614, -40.55615 , -39.62568 , -39.707634, -35.88653 ,
       -35.98243 , -40.28614 , -41.74624 , -40.879337, -41.84437 ,
       -43.657143, -44.65177 , -44.115776, -44.275055, -43.981037,
       -44.237938, -43.687973, -43.760662, -43.3976  , -42.919567,
       -42.142727, -42.42853 , -42.377666, -42.395542, -41.28829 ,
       -41.574146, -42.4205  , -41.430473, -41.15706 , -41.719154,
       -41.485535, -42.346863, -42.57032 , -43.071228, -42.28202 ,
       -41.79384 , -41.57667 , -41.99697 , -41.401665, -40.87029 ,
       -40.962864, -40.4783  , -40.42449 , -40.43128 , -40.65647 ,
       -40.47194 , -40.122643, -40.470573, -40.36758 , -40.06509 ,
       -39.04172 , -38.513466, -38.341705, -38.06644 , -38.18418 ,
       -38.46058 , -38.233692, -38.828243, -40.33467 , -41.545807,
       -41.532658, -40.47445 , -38.36278 , -37.622795, -38.441

'id'

(2028,)

'XC133548.mp3'

## **Random Forest Model**

Saving evaluation results

In [16]:
def evaluate_model(model, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = model.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = model.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [17]:
train_results = {}
val_results = {}

val_scores = {}

### **With all the features**

In [18]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)

training_features.shape

(6543, 244)

In [19]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)

validation_features.shape

(2028, 244)

Fit the model with training data

In [20]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [21]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['all_features'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['all_features'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3806706114398422


In [22]:
val_scores['all_features'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.50      0.66      0.57       126
           1       0.37      0.59      0.45       205
           2       0.22      0.48      0.30        96
           3       0.55      0.74      0.63       152
           4       0.03      0.03      0.03        33
           5       0.42      0.10      0.17        48
           6       0.11      0.11      0.11        38
           7       0.48      0.56      0.52       107
           8       0.15      0.42      0.22        24
           9       0.23      0.44      0.30        32
          10       0.25      0.22      0.23        69
          11       0.00      0.00      0.00       272
          12       0.37      0.26      0.31       203
          13       0.37      0.21      0.27        76
          14       0.04      0.10      0.06        20
          15       0.25      0.38      0.30       119
          16       0.94      0.93      0.94   

### **With MFCCs only**

In [23]:
training_features = train_features['mfcc']

training_features.shape

(6543, 20)

In [24]:
validation_features = val_features['mfcc']

validation_features.shape

(2028, 20)

In [25]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [26]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3141025641025641


In [27]:
val_scores['mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.45      0.64      0.53       126
           1       0.40      0.63      0.49       205
           2       0.24      0.47      0.32        96
           3       0.58      0.81      0.68       152
           4       0.09      0.03      0.05        33
           5       0.00      0.00      0.00        48
           6       0.11      0.13      0.12        38
           7       0.41      0.46      0.43       107
           8       0.27      0.46      0.34        24
           9       0.11      0.44      0.18        32
          10       0.34      0.43      0.38        69
          11       0.36      0.02      0.03       272
          12       0.24      0.11      0.15       203
          13       0.24      0.28      0.25        76
          14       0.07      0.15      0.10        20
          15       0.16      0.31      0.21       119
          16       0.17      0.06      0.09   

### **With Mel-Spectrogram Only**

In [28]:
training_features = train_features['melspectrogram']

training_features.shape

(6543, 128)

In [29]:
validation_features = val_features['melspectrogram']

validation_features.shape

(2028, 128)

In [30]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [31]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3022682445759369


In [32]:
val_scores['melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.45      0.53      0.49       126
           1       0.39      0.58      0.47       205
           2       0.41      0.42      0.41        96
           3       0.31      0.72      0.43       152
           4       0.00      0.00      0.00        33
           5       0.18      0.06      0.09        48
           6       0.07      0.05      0.06        38
           7       0.34      0.42      0.38       107
           8       0.11      0.38      0.17        24
           9       0.10      0.31      0.15        32
          10       0.34      0.17      0.23        69
          11       0.08      0.00      0.01       272
          12       0.31      0.18      0.23       203
          13       0.53      0.24      0.33        76
          14       0.02      0.05      0.03        20
          15       0.25      0.41      0.31       119
          16       0.61      0.17      0.27   

## **With Chroma Only**

In [33]:
training_features = train_features['chroma']

training_features.shape

(6543, 12)

In [34]:
validation_features = val_features['chroma']

validation_features.shape

(2028, 12)

In [35]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [36]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma'] = model.score(validation_features, val_y)

Training accuracy: 0.9926639156350298
Validation accuracy: 0.22386587771203156


In [37]:
val_scores['chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.14      0.44      0.21       126
           1       0.32      0.42      0.36       205
           2       0.10      0.38      0.15        96
           3       0.58      0.57      0.57       152
           4       0.00      0.00      0.00        33
           5       1.00      0.15      0.25        48
           6       0.00      0.00      0.00        38
           7       0.18      0.25      0.21       107
           8       0.05      0.04      0.04        24
           9       0.13      0.22      0.17        32
          10       0.05      0.04      0.05        69
          11       0.13      0.01      0.01       272
          12       0.46      0.13      0.20       203
          13       0.08      0.03      0.04        76
          14       0.02      0.05      0.03        20
          15       0.31      0.44      0.36       119
          16       0.30      0.16      0.21   

## **With CQT Only**

In [38]:
training_features = train_features['cqt']

training_features.shape

(6543, 84)

In [39]:
validation_features = val_features['cqt']

validation_features.shape

(2028, 84)

In [40]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [41]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.22287968441814596


In [42]:
val_scores['cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.12      0.20      0.15       126
           1       0.39      0.49      0.44       205
           2       0.26      0.43      0.32        96
           3       0.16      0.30      0.21       152
           4       0.00      0.00      0.00        33
           5       0.14      0.06      0.09        48
           6       0.04      0.03      0.03        38
           7       0.39      0.61      0.47       107
           8       0.18      0.33      0.23        24
           9       0.21      0.41      0.27        32
          10       0.23      0.23      0.23        69
          11       0.00      0.00      0.00       272
          12       0.20      0.11      0.15       203
          13       0.08      0.07      0.07        76
          14       0.00      0.00      0.00        20
          15       0.14      0.21      0.17       119
          16       0.59      0.44      0.50   

## **With MFCCs and Mel-Spectrogram**

In [43]:
training_features = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)

training_features.shape

(6543, 148)

In [44]:
validation_features = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)

validation_features.shape

(2028, 148)

In [45]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [46]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3145956607495069


In [47]:
val_scores['mfcc_melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.54      0.63      0.58       126
           1       0.38      0.63      0.47       205
           2       0.22      0.44      0.30        96
           3       0.52      0.76      0.61       152
           4       0.04      0.03      0.03        33
           5       0.00      0.00      0.00        48
           6       0.11      0.11      0.11        38
           7       0.29      0.36      0.32       107
           8       0.12      0.42      0.19        24
           9       0.09      0.41      0.14        32
          10       0.20      0.13      0.16        69
          11       0.00      0.00      0.00       272
          12       0.33      0.23      0.27       203
          13       0.46      0.21      0.29        76
          14       0.05      0.10      0.06        20
          15       0.29      0.38      0.33       119
          16       0.52      0.13      0.21   

## **With MFCCs and Chroma**

In [48]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)

training_features.shape

(6543, 32)

In [49]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)

validation_features.shape

(2028, 32)

In [50]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [51]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.38461538461538464


In [52]:
val_scores['mfcc_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.45      0.72      0.56       126
           1       0.43      0.74      0.54       205
           2       0.35      0.55      0.43        96
           3       0.63      0.81      0.71       152
           4       0.00      0.00      0.00        33
           5       0.06      0.02      0.03        48
           6       0.06      0.05      0.06        38
           7       0.47      0.47      0.47       107
           8       0.25      0.38      0.30        24
           9       0.20      0.41      0.27        32
          10       0.39      0.51      0.44        69
          11       0.29      0.02      0.03       272
          12       0.39      0.26      0.31       203
          13       0.31      0.26      0.28        76
          14       0.08      0.20      0.11        20
          15       0.19      0.42      0.26       119
          16       0.76      0.50      0.61   

## **With MFCCs and CQT**

In [53]:
training_features = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)

training_features.shape

(6543, 104)

In [54]:
validation_features = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)

validation_features.shape

(2028, 104)

In [55]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [56]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.34911242603550297


In [57]:
val_scores['mfcc_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.39      0.48      0.43       126
           1       0.32      0.57      0.41       205
           2       0.39      0.51      0.44        96
           3       0.52      0.78      0.62       152
           4       0.00      0.00      0.00        33
           5       0.14      0.06      0.09        48
           6       0.04      0.03      0.03        38
           7       0.34      0.50      0.40       107
           8       0.24      0.50      0.32        24
           9       0.19      0.44      0.27        32
          10       0.31      0.29      0.30        69
          11       0.09      0.00      0.01       272
          12       0.33      0.24      0.27       203
          13       0.26      0.11      0.15        76
          14       0.07      0.20      0.10        20
          15       0.21      0.33      0.25       119
          16       0.91      0.82      0.86   

## **With Chroma and CQT**

In [58]:
training_features = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)

training_features.shape

(6543, 96)

In [59]:
validation_features = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)

validation_features.shape

(2028, 96)

In [60]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [61]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2785996055226824


In [62]:
val_scores['chroma_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.18      0.29      0.22       126
           1       0.47      0.52      0.49       205
           2       0.27      0.49      0.35        96
           3       0.22      0.38      0.28       152
           4       0.00      0.00      0.00        33
           5       0.17      0.08      0.11        48
           6       0.03      0.03      0.03        38
           7       0.35      0.55      0.43       107
           8       0.14      0.33      0.20        24
           9       0.25      0.44      0.32        32
          10       0.25      0.19      0.21        69
          11       0.00      0.00      0.00       272
          12       0.26      0.15      0.19       203
          13       0.77      0.13      0.22        76
          14       0.00      0.00      0.00        20
          15       0.14      0.25      0.18       119
          16       0.78      0.95      0.86   

## **With Mel-Spectrogram and Chroma**

In [63]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)

training_features.shape

(6543, 140)

In [64]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)

validation_features.shape

(2028, 140)

In [65]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [66]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3160749506903353


In [67]:
val_scores['melspectrogram_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.38      0.50      0.43       126
           1       0.44      0.56      0.49       205
           2       0.42      0.43      0.42        96
           3       0.32      0.75      0.45       152
           4       0.00      0.00      0.00        33
           5       0.17      0.06      0.09        48
           6       0.17      0.11      0.13        38
           7       0.39      0.50      0.44       107
           8       0.14      0.46      0.22        24
           9       0.09      0.41      0.15        32
          10       0.19      0.06      0.09        69
          11       0.00      0.00      0.00       272
          12       0.46      0.23      0.31       203
          13       0.45      0.24      0.31        76
          14       0.03      0.10      0.05        20
          15       0.23      0.43      0.30       119
          16       0.47      0.24      0.32   

## **With Mel-Spectrogram and CQT**

In [68]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)

training_features.shape

(6543, 212)

In [69]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)

validation_features.shape

(2028, 212)

In [70]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [71]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3333333333333333


In [72]:
val_scores['melspectrogram_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.40      0.50      0.45       126
           1       0.35      0.52      0.41       205
           2       0.36      0.44      0.40        96
           3       0.30      0.69      0.41       152
           4       0.02      0.03      0.03        33
           5       0.21      0.10      0.14        48
           6       0.14      0.11      0.12        38
           7       0.46      0.47      0.47       107
           8       0.12      0.42      0.19        24
           9       0.15      0.34      0.21        32
          10       0.34      0.19      0.24        69
          11       0.09      0.00      0.01       272
          12       0.31      0.20      0.25       203
          13       0.36      0.13      0.19        76
          14       0.00      0.00      0.00        20
          15       0.25      0.39      0.31       119
          16       0.93      0.97      0.95   

## **With Mel-Spectrogram, MFCCs, CQT**

In [73]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)

training_features.shape

(6543, 232)

In [74]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)

validation_features.shape

(2028, 232)

In [75]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [76]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.36587771203155817


In [77]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.50      0.68      0.58       126
           1       0.38      0.58      0.46       205
           2       0.23      0.47      0.31        96
           3       0.51      0.69      0.59       152
           4       0.08      0.06      0.07        33
           5       0.75      0.12      0.21        48
           6       0.06      0.05      0.06        38
           7       0.37      0.47      0.41       107
           8       0.14      0.42      0.21        24
           9       0.27      0.44      0.33        32
          10       0.23      0.22      0.22        69
          11       0.00      0.00      0.00       272
          12       0.26      0.20      0.23       203
          13       0.40      0.21      0.28        76
          14       0.04      0.10      0.06        20
          15       0.28      0.38      0.32       119
          16       0.92      0.93      0.92   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [78]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)

training_features.shape

(6543, 160)

In [79]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)

validation_features.shape

(2028, 160)

In [80]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [81]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.34714003944773175


In [82]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.51      0.65      0.57       126
           1       0.38      0.65      0.48       205
           2       0.20      0.45      0.27        96
           3       0.62      0.78      0.69       152
           4       0.04      0.03      0.04        33
           5       0.00      0.00      0.00        48
           6       0.09      0.08      0.08        38
           7       0.44      0.57      0.49       107
           8       0.15      0.50      0.24        24
           9       0.13      0.41      0.20        32
          10       0.31      0.22      0.25        69
          11       0.12      0.01      0.01       272
          12       0.36      0.30      0.33       203
          13       0.35      0.20      0.25        76
          14       0.05      0.15      0.08        20
          15       0.30      0.40      0.35       119
          16       0.50      0.20      0.28   

# Review results from all models

In [83]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.000,0.381
3,chroma,0.993,0.224
8,chroma_cqt,1.000,0.279
4,cqt,1.000,0.223
2,melspectrogram,1.000,0.302
9,melspectrogram_chroma,1.000,0.316
12,melspectrogram_chroma_mfcc,1.000,0.347
10,melspectrogram_cqt,1.000,0.333
11,melspectrogram_cqt_mfcc,1.000,0.366
1,mfcc,1.000,0.314


In [84]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(3)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features     0.350      0.837
3                    chroma     0.206      0.741
8                chroma_cqt     0.251      0.784
4                       cqt     0.201      0.731
2            melspectrogram     0.275      0.783
9     melspectrogram_chroma     0.290      0.799
10       melspectrogram_cqt     0.303      0.791
11  melspectrogram_cqt_mfcc     0.315      0.832
1                      mfcc     0.274      0.803
6               mfcc_chroma     0.350      0.840
7                  mfcc_cqt     0.315      0.811
5       mfcc_melspectrogram     0.282      0.809
